# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [8]:
from pyspark.sql import SparkSession

session = SparkSession.builder.getOrCreate()

Within the SparkSession there is a SpakContext that we can use to, for example, create RDDs:

In [9]:
session.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

#### Passing other options to spark session:
    
    

In [10]:
session = SparkSession.builder\
                .config('someoption', 'somevalue')\
                .config('anotheroption', 'anothervalue')\
                .getOrCreate()

We can check option values in the resulting session like this:

In [14]:
session.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1539960611002'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('anotheroption', 'anothervalue'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('someoption', 'somevalue'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '33643'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '10.2.28.80')]

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [39]:
import random

random.choice(['widgeteer', 'smoke salesman', 'wizard', 'psycopath'])

'psycopath'

In [129]:
random.seed(42)

ids = range(20)
positions = [random.choice(['widgeteer', 'smoke salesman', 'wizard', 'psychopath']) for id in ids ]
positions

['widgeteer',
 'widgeteer',
 'wizard',
 'smoke salesman',
 'smoke salesman',
 'smoke salesman',
 'widgeteer',
 'widgeteer',
 'psychopath',
 'widgeteer',
 'widgeteer',
 'widgeteer',
 'smoke salesman',
 'smoke salesman',
 'widgeteer',
 'smoke salesman',
 'psychopath',
 'smoke salesman',
 'psychopath',
 'wizard']

A data frame can be created like this:

In [43]:
rows = zip(ids, positions)

df = session.createDataFrame(rows)
df

DataFrame[_1: bigint, _2: string]

But it won't have informative field names.

In [44]:
df.take(5)

[Row(_1=0, _2='widgeteer'),
 Row(_1=1, _2='widgeteer'),
 Row(_1=2, _2='wizard'),
 Row(_1=3, _2='smoke salesman'),
 Row(_1=4, _2='smoke salesman')]

In [45]:
df.show(5)

+---+--------------+
| _1|            _2|
+---+--------------+
|  0|     widgeteer|
|  1|     widgeteer|
|  2|        wizard|
|  3|smoke salesman|
|  4|smoke salesman|
+---+--------------+
only showing top 5 rows



In [47]:
from pyspark.sql import Row

help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <Row(name, age)>
 |  >>> 'name' 

In [48]:
row = df.first()
row

Row(_1=0, _2='widgeteer')

In [49]:
row._1

0

We can access fields in two ways, like in Pandas:

In [50]:
row._2

'widgeteer'

In [51]:
row['_2']

'widgeteer'

In [53]:
rows = zip(ids, positions)
df = session.createDataFrame(rows, schema=['id_number', 'position'])
df.show(5)

+---------+--------------+
|id_number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
+---------+--------------+
only showing top 5 rows



In [54]:
df.printSchema()

root
 |-- id_number: long (nullable = true)
 |-- position: string (nullable = true)



In [55]:
df.rdd

MapPartitionsRDD[33] at javaToPython at NativeMethodAccessorImpl.java:0

In [56]:
df.rdd.take(5)

[Row(id_number=0, position='widgeteer'),
 Row(id_number=1, position='widgeteer'),
 Row(id_number=2, position='wizard'),
 Row(id_number=3, position='smoke salesman'),
 Row(id_number=4, position='smoke salesman')]

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [57]:
lines = session.sparkContext.textFile('coupon150720.csv')
lines

coupon150720.csv MapPartitionsRDD[37] at textFile at NativeMethodAccessorImpl.java:0

In [58]:
lines.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [70]:
def parse(line):

    fields = line.split(',')
    coupon = (fields[0], fields[2], fields[3], fields[4], float(fields[6]))
    
    return coupon # represented as a 5-element tuple

coupons = lines.map(parse)

coupons.take(5)

[('79062005698500', 'MAA', 'AUH', '9W', 56.79),
 ('79062005698500', 'AUH', 'CDG', '9W', 84.34),
 ('79062005924069', 'CJB', 'MAA', '9W', 60.0),
 ('79065668570385', 'DEL', 'DXB', '9W', 160.63),
 ('79065668737021', 'AUH', 'IXE', '9W', 152.46)]

In [75]:
lines.count()

1232662

In [76]:
type(lines)

pyspark.rdd.RDD

In [73]:
session

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [80]:
df_from_csv = session.read.csv('coupon150720.csv')
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [82]:
df_from_csv = session.sql('SELECT * from csv.`coupon150720.csv`')
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [84]:
df_from_csv = session.sql('SELECT _c0, _c2, _c3, _c4, _c5, _c6 from csv.`coupon150720.csv`')
df_from_csv.show(5)
df_from_csv.printSchema()

+--------------+---+---+---+---+------+
|           _c0|_c2|_c3|_c4|_c5|   _c6|
+--------------+---+---+---+---+------+
|79062005698500|MAA|AUH| 9W| 9W| 56.79|
|79062005698500|AUH|CDG| 9W| 9W| 84.34|
|79062005924069|CJB|MAA| 9W| 9W|  60.0|
|79065668570385|DEL|DXB| 9W| 9W|160.63|
|79065668737021|AUH|IXE| 9W| 9W|152.46|
+--------------+---+---+---+---+------+
only showing top 5 rows

root
 |-- _c0: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



In [87]:
df_from_csv = session.sql('SELECT _c0, _c2, _c3, _c4, _c5, CAST(_c6 AS FLOAT)  from csv.`coupon150720.csv`')
df_from_csv.show(5)
df_from_csv.printSchema()

+--------------+---+---+---+---+------+
|           _c0|_c2|_c3|_c4|_c5|   _c6|
+--------------+---+---+---+---+------+
|79062005698500|MAA|AUH| 9W| 9W| 56.79|
|79062005698500|AUH|CDG| 9W| 9W| 84.34|
|79062005924069|CJB|MAA| 9W| 9W|  60.0|
|79065668570385|DEL|DXB| 9W| 9W|160.63|
|79065668737021|AUH|IXE| 9W| 9W|152.46|
+--------------+---+---+---+---+------+
only showing top 5 rows

root
 |-- _c0: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: float (nullable = true)



### Inferring and specifying schemas

In [130]:
from pyspark.sql import types


types.IntegerType()

IntegerType

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [131]:
employee_schema = types.StructType([types.StructField('id_number', types.LongType(), nullable=False),
                                    types.StructField('position', types.StringType(), nullable=True)])

employees = session.createDataFrame(zip(ids, positions), schema=employee_schema) 
employees.printSchema()

root
 |-- id_number: long (nullable = false)
 |-- position: string (nullable = true)



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [132]:
session.read.parquet
session.read.json
session.read.jdbc

<bound method DataFrameReader.jdbc of <pyspark.sql.readwriter.DataFrameReader object at 0x7f8f812d3f28>>

### Basic operations with DataFrames

In [133]:
employees.show(5)

+---------+--------------+
|id_number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
+---------+--------------+
only showing top 5 rows



In [134]:
employee_0 = employees.first()

employee_0['id_number']



0

### Filtering and selecting

Syntax inspired in SQL.

In [135]:
employees.select('id_number')

DataFrame[id_number: bigint]

If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [136]:
type(employees['id_number'])

pyspark.sql.column.Column

In [137]:
employees.filter(employees['id_number'] < 5).show()

+---------+--------------+
|id_number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
+---------+--------------+



That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

In [138]:
employees.filter('id_number' < 5).show()

TypeError: '<' not supported between instances of 'str' and 'int'

`where` is exactly synonimous with `filter`

In [139]:
employees.where(employees['id_number'] < 5).show()

+---------+--------------+
|id_number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
+---------+--------------+



A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

In [140]:
col = employees['id_number']
col.collect()

TypeError: 'Column' object is not callable

#### Exercise

Extract all employee ids which correspond to pyschopaths

In [141]:
employees.show()

+---------+--------------+
|id_number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
|        5|smoke salesman|
|        6|     widgeteer|
|        7|     widgeteer|
|        8|    psychopath|
|        9|     widgeteer|
|       10|     widgeteer|
|       11|     widgeteer|
|       12|smoke salesman|
|       13|smoke salesman|
|       14|     widgeteer|
|       15|smoke salesman|
|       16|    psychopath|
|       17|smoke salesman|
|       18|    psychopath|
|       19|        wizard|
+---------+--------------+



In [142]:
employees.filter(employees['position'] == 'psychopath').select('id_number').show()

+---------+
|id_number|
+---------+
|        8|
|       16|
|       18|
+---------+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [147]:
employees['salary'] = employees['id_number'] ** 2

TypeError: 'DataFrame' object does not support item assignment

In [414]:
employees.withColumn('square', employees['id_number'] ** 2)\
         .withColumn('triquitri', employees['id_number'] % 2)\
         .withColumn('trocotro', employees['id_number'] % 2)   

DataFrame[id_number: bigint, position: string, square: double, triquitri: bigint, trocotro: bigint]

In [149]:
employees

DataFrame[id_number: bigint, position: string]

In [150]:
employees.select('id_number',
                 'position',
                 employees['id_number'] ** 2)

DataFrame[id_number: bigint, position: string, POWER(id_number, 2): double]

In [151]:
employees.select('id_number',
                 'position',
                 (employees['id_number'] ** 2).alias('holi'))

DataFrame[id_number: bigint, position: string, holi: double]

### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [153]:
from pyspark.sql import functions

help(functions.log1p)

Help on function log1p in module pyspark.sql.functions:

log1p(col)
    Computes the natural logarithm of the given value plus one.
    
    .. versionadded:: 1.4



In [160]:
df = employees.select('id_number',
                      'position',
                      functions.log1p(employees['id_number']))

df.show(5)

+---------+--------------+------------------+
|id_number|      position|  LOG1P(id_number)|
+---------+--------------+------------------+
|        0|     widgeteer|               0.0|
|        1|     widgeteer|0.6931471805599453|
|        2|        wizard|1.0986122886681096|
|        3|smoke salesman|1.3862943611198906|
|        4|smoke salesman|1.6094379124341003|
+---------+--------------+------------------+
only showing top 5 rows



In [161]:
df = employees.select('id_number',
                      'position',
                      functions.log1p('id_number'))

df.show(5)

+---------+--------------+------------------+
|id_number|      position|  LOG1P(id_number)|
+---------+--------------+------------------+
|        0|     widgeteer|               0.0|
|        1|     widgeteer|0.6931471805599453|
|        2|        wizard|1.0986122886681096|
|        3|smoke salesman|1.3862943611198906|
|        4|smoke salesman|1.6094379124341003|
+---------+--------------+------------------+
only showing top 5 rows



In [165]:
import math

math.log1p(0)

0.0

In [168]:
df = employees.select('id_number',
                      'position',
                      math.log1p(employees['id_number']))

df.show(5)

TypeError: must be real number, not Column

This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

But we can transform it into a udf:

In [173]:
udf_log1p = functions.udf(math.log1p) 

df = employees.select('id_number',
                      'position',
                      udf_log1p(employees['id_number']))

df.show(5)

+---------+--------------+------------------+
|id_number|      position|  log1p(id_number)|
+---------+--------------+------------------+
|        0|     widgeteer|               0.0|
|        1|     widgeteer|0.6931471805599453|
|        2|        wizard|1.0986122886681096|
|        3|smoke salesman|1.3862943611198906|
|        4|smoke salesman|1.6094379124341003|
+---------+--------------+------------------+
only showing top 5 rows



In [175]:
df.select('position').distinct().show()

+--------------+
|      position|
+--------------+
|    psychopath|
|     widgeteer|
|smoke salesman|
|        wizard|
+--------------+



We can do the same with any function we dream up:

In [178]:
shorten = functions.udf(lambda word: word[:3])

df = employees.select('id_number',
                      'position',
                      shorten('position').alias('short_pos'))

df.show(5)

+---------+--------------+---------+
|id_number|      position|short_pos|
+---------+--------------+---------+
|        0|     widgeteer|      wid|
|        1|     widgeteer|      wid|
|        2|        wizard|      wiz|
|        3|smoke salesman|      smo|
|        4|smoke salesman|      smo|
+---------+--------------+---------+
only showing top 5 rows



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [181]:
udf_log1p = functions.udf(math.log1p) 

df = employees.select('id_number',
                      'position',
                      udf_log1p(employees['id_number']))

df.printSchema()
df.show(2)

root
 |-- id_number: long (nullable = false)
 |-- position: string (nullable = true)
 |-- log1p(id_number): string (nullable = true)

+---------+---------+------------------+
|id_number| position|  log1p(id_number)|
+---------+---------+------------------+
|        0|widgeteer|               0.0|
|        1|widgeteer|0.6931471805599453|
+---------+---------+------------------+
only showing top 2 rows



In [180]:
udf_log1p.returnType

StringType

In [182]:
help(functions.udf)

Help on function udf in module pyspark.sql.functions:

udf(f=None, returnType=StringType)
    Creates a user defined function (UDF).
    
    .. note:: The user-defined functions are considered deterministic by default. Due to
        optimization, duplicate invocations may be eliminated or the function may even be invoked
        more times than it is present in the query. If your function is not deterministic, call
        `asNondeterministic` on the user defined function. E.g.:
    
    >>> from pyspark.sql.types import IntegerType
    >>> import random
    >>> random_udf = udf(lambda: int(random.random() * 100), IntegerType()).asNondeterministic()
    
    .. note:: The user-defined functions do not support conditional expressions or short circuiting
        in boolean expressions and it ends up with being executed all internally. If the functions
        can fail on special rows, the workaround is to incorporate the condition into the functions.
    
    :param f: python function 

Think about this function: what is its return type?

In [ ]:
def anonymous(element):
    result = element + element
    return result

In [186]:
udf_log1p_2 = functions.udf(math.log1p, returnType=types.FloatType())
udf_log1p_2.returnType

FloatType

#### Exercise: 

Create a 'salary' field in our df. make it 100000 for psychopaths, 35000 for widgeteers, 50000 for smoke salesmen 60000 for wizards



In [187]:
df.select('position').distinct().show()

+--------------+
|      position|
+--------------+
|    psychopath|
|     widgeteer|
|smoke salesman|
|        wizard|
+--------------+



In [223]:
def salary(position):

    salary_tables = {'psychopath' : 100000, 'widgeteer' : 35000, 'smoke salesman' : 50000, 'wizard' : 60000 }
    
    return salary_tables[position]
    
salary('wizard')

60000

In [224]:
salary_udf = functions.udf(salary) 

df = employees.select('id_number',
                      'position',
                      salary_udf(employees['position']))
df.show(5)

+---------+--------------+----------------+
|id_number|      position|salary(position)|
+---------+--------------+----------------+
|        0|     widgeteer|           35000|
|        1|     widgeteer|           35000|
|        2|        wizard|           60000|
|        3|smoke salesman|           50000|
|        4|smoke salesman|           50000|
+---------+--------------+----------------+
only showing top 5 rows



In [225]:
df = df.withColumn('bonus', df['salary(position)'] * 0.1) 

df.printSchema()

root
 |-- id_number: long (nullable = false)
 |-- position: string (nullable = true)
 |-- salary(position): string (nullable = true)
 |-- bonus: double (nullable = true)



In [226]:
salary_udf = functions.udf(salary, returnType=types.IntegerType()) 

df = employees.select('id_number',
                      'position',
                      salary_udf(employees['position']))
df.printSchema()

root
 |-- id_number: long (nullable = false)
 |-- position: string (nullable = true)
 |-- salary(position): integer (nullable = true)



If we have a column that is not the desired type, we can convert it with `cast`.

In [227]:
df.select('id_number',
          'position',
          df['salary(position)'].cast(types.IntegerType())
         )

DataFrame[id_number: bigint, position: string, salary(position): int]

### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [228]:
df.stat.corr('id_number', 'salary(position)')

0.38648340657639785

In [229]:
df.stat.cov('id_number', 'salary(position)')

50789.47368421053

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [232]:
location_udf = functions.udf(lambda: random.choice(['Madrid', 'Barcelona']))

with_locs = df.select('id_number',
               'position',
               df['salary(position)'].alias('salary'),
               location_udf().alias('location'))

In [260]:
with_locs.cache()
with_locs.show()

+---------+--------------+------+---------+
|id_number|      position|salary| location|
+---------+--------------+------+---------+
|        0|     widgeteer| 35000|   Madrid|
|        1|     widgeteer| 35000|Barcelona|
|        2|        wizard| 60000|Barcelona|
|        3|smoke salesman| 50000|   Madrid|
|        4|smoke salesman| 50000|   Madrid|
|        5|smoke salesman| 50000|Barcelona|
|        6|     widgeteer| 35000|   Madrid|
|        7|     widgeteer| 35000|Barcelona|
|        8|    psychopath|100000|   Madrid|
|        9|     widgeteer| 35000|Barcelona|
|       10|     widgeteer| 35000|   Madrid|
|       11|     widgeteer| 35000|Barcelona|
|       12|smoke salesman| 50000|   Madrid|
|       13|smoke salesman| 50000|Barcelona|
|       14|     widgeteer| 35000|   Madrid|
|       15|smoke salesman| 50000|Barcelona|
|       16|    psychopath|100000|   Madrid|
|       17|smoke salesman| 50000|Barcelona|
|       18|    psychopath|100000|   Madrid|
|       19|        wizard| 60000

In [262]:
with_locs.crosstab('position', 'location').show()

+-----------------+---------+------+
|position_location|Barcelona|Madrid|
+-----------------+---------+------+
|   smoke salesman|        4|     3|
|       psychopath|        0|     3|
|           wizard|        2|     0|
|        widgeteer|        4|     4|
+-----------------+---------+------+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [264]:
gb = with_locs.groupby('location')
gb

GroupedData has several aggregation functions defined:

In [266]:
gb.avg().show()

+---------+--------------+-----------+
| location|avg(id_number)|avg(salary)|
+---------+--------------+-----------+
|   Madrid|           9.1|    59000.0|
|Barcelona|           9.9|    46000.0|
+---------+--------------+-----------+



In [270]:
gb.avg('salary').show()

+---------+-----------+
| location|avg(salary)|
+---------+-----------+
|   Madrid|    59000.0|
|Barcelona|    46000.0|
+---------+-----------+



We can do several aggregations in a single step, with a number of different syntaxes:

In [272]:
gb.agg({'id_number': 'max', 'salary': 'mean' }).show()

+---------+--------------+-----------+
| location|max(id_number)|avg(salary)|
+---------+--------------+-----------+
|   Madrid|            18|    59000.0|
|Barcelona|            19|    46000.0|
+---------+--------------+-----------+



In [274]:
result = gb.agg(functions.mean('salary'),
                functions.count('salary'),
                functions.countDistinct('position'))

result.show()

+---------+-----------+-------------+------------------------+
| location|avg(salary)|count(salary)|count(DISTINCT position)|
+---------+-----------+-------------+------------------------+
|   Madrid|    59000.0|           10|                       3|
|Barcelona|    46000.0|           10|                       3|
+---------+-----------+-------------+------------------------+



### Intersections

Very much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [306]:
random.seed(42)

raises = session.createDataFrame(list(zip([random.choice(['Barcelona', 'Sevilla', 'Madrid']) for _ in range(7)],
                                           [random.randint(0, 19) for _ in range(7)],
                                           [random.random() * 10000 for _ in range(7)])),
                                  schema= ['position', 'id_number', 'raise']).cache()
raises.show()

+---------+---------+------------------+
| position|id_number|             raise|
+---------+---------+------------------+
|   Madrid|        4|297.97219438070346|
|Barcelona|        3|2186.3797480360336|
|Barcelona|       17| 5053.552881033624|
|   Madrid|        2|265.35969683863624|
|  Sevilla|       18| 1988.376506866485|
|Barcelona|       13|6498.8443777952325|
|Barcelona|        1| 5449.414806032167|
+---------+---------+------------------+



In [321]:
with_locs.join(raises).show() 

AnalysisException: 'Detected cartesian product for INNER join between logical plans\nInMemoryRelation [id_number#612L, position#613, salary#1083, location#1084], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n   +- *(1) Project [id_number#612L, position#613, pythonUDF0#1267 AS salary#1083, pythonUDF1#1268 AS location#1084]\n      +- BatchEvalPython [salary(position#613), <lambda>()], [id_number#612L, position#613, pythonUDF0#1267, pythonUDF1#1268]\n         +- Scan ExistingRDD[id_number#612L,position#613]\nand\nInMemoryRelation [position#2243, id_number#2244L, raise#2245], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n   +- Scan ExistingRDD[position#2243,id_number#2244L,raise#2245]\nJoin condition is missing or trivial.\nUse the CROSS JOIN syntax to allow cartesian products between these relations.;'

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [310]:
with_locs.join(raises, on='id_number').show() 

+---------+--------------+------+---------+---------+------------------+
|id_number|      position|salary| location| position|             raise|
+---------+--------------+------+---------+---------+------------------+
|        1|     widgeteer| 35000|Barcelona|Barcelona| 5449.414806032167|
|        2|        wizard| 60000|Barcelona|   Madrid|265.35969683863624|
|        3|smoke salesman| 50000|   Madrid|Barcelona|2186.3797480360336|
|        4|smoke salesman| 50000|   Madrid|   Madrid|297.97219438070346|
|       13|smoke salesman| 50000|Barcelona|Barcelona|6498.8443777952325|
|       17|smoke salesman| 50000|Barcelona|Barcelona| 5053.552881033624|
|       18|    psychopath|100000|   Madrid|  Sevilla| 1988.376506866485|
+---------+--------------+------+---------+---------+------------------+



In [311]:
with_locs.join(raises, on='id_number',how='left').show() 

+---------+--------------+------+---------+---------+------------------+
|id_number|      position|salary| location| position|             raise|
+---------+--------------+------+---------+---------+------------------+
|        0|     widgeteer| 35000|   Madrid|     null|              null|
|        1|     widgeteer| 35000|Barcelona|Barcelona| 5449.414806032167|
|        2|        wizard| 60000|Barcelona|   Madrid|265.35969683863624|
|        3|smoke salesman| 50000|   Madrid|Barcelona|2186.3797480360336|
|        4|smoke salesman| 50000|   Madrid|   Madrid|297.97219438070346|
|        5|smoke salesman| 50000|Barcelona|     null|              null|
|        6|     widgeteer| 35000|   Madrid|     null|              null|
|        7|     widgeteer| 35000|Barcelona|     null|              null|
|        8|    psychopath|100000|   Madrid|     null|              null|
|        9|     widgeteer| 35000|Barcelona|     null|              null|
|       10|     widgeteer| 35000|   Madrid|     nul

In [312]:
with_locs.join(raises, on=['id_number', 'position'],how='left').show() 

+---------+--------------+------+---------+-----+
|id_number|      position|salary| location|raise|
+---------+--------------+------+---------+-----+
|        0|     widgeteer| 35000|   Madrid| null|
|        1|     widgeteer| 35000|Barcelona| null|
|        2|        wizard| 60000|Barcelona| null|
|        3|smoke salesman| 50000|   Madrid| null|
|        4|smoke salesman| 50000|   Madrid| null|
|        5|smoke salesman| 50000|Barcelona| null|
|        6|     widgeteer| 35000|   Madrid| null|
|        7|     widgeteer| 35000|Barcelona| null|
|        8|    psychopath|100000|   Madrid| null|
|        9|     widgeteer| 35000|Barcelona| null|
|       10|     widgeteer| 35000|   Madrid| null|
|       11|     widgeteer| 35000|Barcelona| null|
|       12|smoke salesman| 50000|   Madrid| null|
|       13|smoke salesman| 50000|Barcelona| null|
|       14|     widgeteer| 35000|   Madrid| null|
|       15|smoke salesman| 50000|Barcelona| null|
|       16|    psychopath|100000|   Madrid| null|


In [323]:
joined = with_locs.join(raises, 
                        (with_locs['id_number'] == raises['id_number']) &
                        (with_locs['location'] == raises['position']), 
                         how='left').cache()

joined.show()

+---------+--------------+------+---------+---------+---------+------------------+
|id_number|      position|salary| location| position|id_number|             raise|
+---------+--------------+------+---------+---------+---------+------------------+
|        0|     widgeteer| 35000|   Madrid|     null|     null|              null|
|        1|     widgeteer| 35000|Barcelona|Barcelona|        1| 5449.414806032167|
|        2|        wizard| 60000|Barcelona|     null|     null|              null|
|        3|smoke salesman| 50000|   Madrid|     null|     null|              null|
|        4|smoke salesman| 50000|   Madrid|   Madrid|        4|297.97219438070346|
|        5|smoke salesman| 50000|Barcelona|     null|     null|              null|
|        6|     widgeteer| 35000|   Madrid|     null|     null|              null|
|        7|     widgeteer| 35000|Barcelona|     null|     null|              null|
|        8|    psychopath|100000|   Madrid|     null|     null|              null|
|   

#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

We can erase it with `unpersist`

In [324]:
joined.unpersist()

DataFrame[id_number: bigint, position: string, salary: int, location: string, position: string, id_number: bigint, raise: double]

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

1) Calculate the mean and std of salary for each location

In [347]:
starting_point = with_locs.join(raises, on='id_number',how='left')

stats = starting_point.groupby('location')\
              .agg(functions.mean('salary').alias('avg_salary'),
                   functions.stddev('salary').alias('std_salary'))

stats.show()

+---------+----------+------------------+
| location|avg_salary|        std_salary|
+---------+----------+------------------+
|   Madrid|   59000.0| 29040.20201949934|
|Barcelona|   46000.0|10219.806477837263|
+---------+----------+------------------+



2) Annotate each employee with the stats corresponding to their location

In [350]:
annotated = starting_point.join(stats, on='location', how='left')
annotated.printSchema()

root
 |-- location: string (nullable = true)
 |-- id_number: long (nullable = false)
 |-- position: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- raise: double (nullable = true)
 |-- avg_salary: double (nullable = true)
 |-- std_salary: double (nullable = true)



3) Calculate the z-score

In [356]:
result = annotated.withColumn('zscore', 
                                (annotated['salary'] - annotated['avg_salary']) / annotated['std_salary'])
result.select('id_number', 'zscore').show()

+---------+--------------------+
|id_number|              zscore|
+---------+--------------------+
|        0| -0.8264405317802183|
|        1| -1.0763413205381795|
|        2|  1.3698889534122285|
|        3|-0.30991519941758183|
|        4|-0.30991519941758183|
|        5| 0.39139684383206524|
|        6| -0.8264405317802183|
|        7| -1.0763413205381795|
|        8|  1.4118359084578729|
|        9| -1.0763413205381795|
|       10| -0.8264405317802183|
|       11| -1.0763413205381795|
|       12|-0.30991519941758183|
|       13| 0.39139684383206524|
|       14| -0.8264405317802183|
|       15| 0.39139684383206524|
|       16|  1.4118359084578729|
|       17| 0.39139684383206524|
|       18|  1.4118359084578729|
|       19|  1.3698889534122285|
+---------+--------------------+



### Handling null values

In [358]:
with_locs

DataFrame[id_number: bigint, position: string, salary: int, location: string]

In [373]:
the_others = session.createDataFrame([(100, 'superboss', 12000000, None),
                                      (101, None, 100000, 'Miami'),
                                      (102, None, None, None),
                                      (103, None, None, 'Palencia'),
                                     ],
                                     )

df = with_locs.union(the_others)
df.show(25)

+---------+--------------+--------+---------+
|id_number|      position|  salary| location|
+---------+--------------+--------+---------+
|        0|     widgeteer|   35000|   Madrid|
|        1|     widgeteer|   35000|Barcelona|
|        2|        wizard|   60000|Barcelona|
|        3|smoke salesman|   50000|   Madrid|
|        4|smoke salesman|   50000|   Madrid|
|        5|smoke salesman|   50000|Barcelona|
|        6|     widgeteer|   35000|   Madrid|
|        7|     widgeteer|   35000|Barcelona|
|        8|    psychopath|  100000|   Madrid|
|        9|     widgeteer|   35000|Barcelona|
|       10|     widgeteer|   35000|   Madrid|
|       11|     widgeteer|   35000|Barcelona|
|       12|smoke salesman|   50000|   Madrid|
|       13|smoke salesman|   50000|Barcelona|
|       14|     widgeteer|   35000|   Madrid|
|       15|smoke salesman|   50000|Barcelona|
|       16|    psychopath|  100000|   Madrid|
|       17|smoke salesman|   50000|Barcelona|
|       18|    psychopath|  100000

In [371]:
df.dropna().show(25)

+---------+--------------+------+---------+
|id_number|      position|salary| location|
+---------+--------------+------+---------+
|        0|     widgeteer| 35000|   Madrid|
|        1|     widgeteer| 35000|Barcelona|
|        2|        wizard| 60000|Barcelona|
|        3|smoke salesman| 50000|   Madrid|
|        4|smoke salesman| 50000|   Madrid|
|        5|smoke salesman| 50000|Barcelona|
|        6|     widgeteer| 35000|   Madrid|
|        7|     widgeteer| 35000|Barcelona|
|        8|    psychopath|100000|   Madrid|
|        9|     widgeteer| 35000|Barcelona|
|       10|     widgeteer| 35000|   Madrid|
|       11|     widgeteer| 35000|Barcelona|
|       12|smoke salesman| 50000|   Madrid|
|       13|smoke salesman| 50000|Barcelona|
|       14|     widgeteer| 35000|   Madrid|
|       15|smoke salesman| 50000|Barcelona|
|       16|    psychopath|100000|   Madrid|
|       17|smoke salesman| 50000|Barcelona|
|       18|    psychopath|100000|   Madrid|
|       19|        wizard| 60000

In [375]:
df.dropna(how='all', subset=['salary', 'location']).show(25)

+---------+--------------+--------+---------+
|id_number|      position|  salary| location|
+---------+--------------+--------+---------+
|        0|     widgeteer|   35000|   Madrid|
|        1|     widgeteer|   35000|Barcelona|
|        2|        wizard|   60000|Barcelona|
|        3|smoke salesman|   50000|   Madrid|
|        4|smoke salesman|   50000|   Madrid|
|        5|smoke salesman|   50000|Barcelona|
|        6|     widgeteer|   35000|   Madrid|
|        7|     widgeteer|   35000|Barcelona|
|        8|    psychopath|  100000|   Madrid|
|        9|     widgeteer|   35000|Barcelona|
|       10|     widgeteer|   35000|   Madrid|
|       11|     widgeteer|   35000|Barcelona|
|       12|smoke salesman|   50000|   Madrid|
|       13|smoke salesman|   50000|Barcelona|
|       14|     widgeteer|   35000|   Madrid|
|       15|smoke salesman|   50000|Barcelona|
|       16|    psychopath|  100000|   Madrid|
|       17|smoke salesman|   50000|Barcelona|
|       18|    psychopath|  100000

In [376]:
df.dropna(how='any', subset=['salary', 'location']).show(25)

+---------+--------------+------+---------+
|id_number|      position|salary| location|
+---------+--------------+------+---------+
|        0|     widgeteer| 35000|   Madrid|
|        1|     widgeteer| 35000|Barcelona|
|        2|        wizard| 60000|Barcelona|
|        3|smoke salesman| 50000|   Madrid|
|        4|smoke salesman| 50000|   Madrid|
|        5|smoke salesman| 50000|Barcelona|
|        6|     widgeteer| 35000|   Madrid|
|        7|     widgeteer| 35000|Barcelona|
|        8|    psychopath|100000|   Madrid|
|        9|     widgeteer| 35000|Barcelona|
|       10|     widgeteer| 35000|   Madrid|
|       11|     widgeteer| 35000|Barcelona|
|       12|smoke salesman| 50000|   Madrid|
|       13|smoke salesman| 50000|Barcelona|
|       14|     widgeteer| 35000|   Madrid|
|       15|smoke salesman| 50000|Barcelona|
|       16|    psychopath|100000|   Madrid|
|       17|smoke salesman| 50000|Barcelona|
|       18|    psychopath|100000|   Madrid|
|       19|        wizard| 60000

In [377]:
df.dropna(subset=['salary', 'location', 'position'], thresh=2).show(25)

+---------+--------------+--------+---------+
|id_number|      position|  salary| location|
+---------+--------------+--------+---------+
|        0|     widgeteer|   35000|   Madrid|
|        1|     widgeteer|   35000|Barcelona|
|        2|        wizard|   60000|Barcelona|
|        3|smoke salesman|   50000|   Madrid|
|        4|smoke salesman|   50000|   Madrid|
|        5|smoke salesman|   50000|Barcelona|
|        6|     widgeteer|   35000|   Madrid|
|        7|     widgeteer|   35000|Barcelona|
|        8|    psychopath|  100000|   Madrid|
|        9|     widgeteer|   35000|Barcelona|
|       10|     widgeteer|   35000|   Madrid|
|       11|     widgeteer|   35000|Barcelona|
|       12|smoke salesman|   50000|   Madrid|
|       13|smoke salesman|   50000|Barcelona|
|       14|     widgeteer|   35000|   Madrid|
|       15|smoke salesman|   50000|Barcelona|
|       16|    psychopath|  100000|   Madrid|
|       17|smoke salesman|   50000|Barcelona|
|       18|    psychopath|  100000

In [380]:
df.fillna('holi').show(25)

+---------+--------------+--------+---------+
|id_number|      position|  salary| location|
+---------+--------------+--------+---------+
|        0|     widgeteer|   35000|   Madrid|
|        1|     widgeteer|   35000|Barcelona|
|        2|        wizard|   60000|Barcelona|
|        3|smoke salesman|   50000|   Madrid|
|        4|smoke salesman|   50000|   Madrid|
|        5|smoke salesman|   50000|Barcelona|
|        6|     widgeteer|   35000|   Madrid|
|        7|     widgeteer|   35000|Barcelona|
|        8|    psychopath|  100000|   Madrid|
|        9|     widgeteer|   35000|Barcelona|
|       10|     widgeteer|   35000|   Madrid|
|       11|     widgeteer|   35000|Barcelona|
|       12|smoke salesman|   50000|   Madrid|
|       13|smoke salesman|   50000|Barcelona|
|       14|     widgeteer|   35000|   Madrid|
|       15|smoke salesman|   50000|Barcelona|
|       16|    psychopath|  100000|   Madrid|
|       17|smoke salesman|   50000|Barcelona|
|       18|    psychopath|  100000

In [381]:
df.fillna('casita', subset=['location']).show(25)

+---------+--------------+--------+---------+
|id_number|      position|  salary| location|
+---------+--------------+--------+---------+
|        0|     widgeteer|   35000|   Madrid|
|        1|     widgeteer|   35000|Barcelona|
|        2|        wizard|   60000|Barcelona|
|        3|smoke salesman|   50000|   Madrid|
|        4|smoke salesman|   50000|   Madrid|
|        5|smoke salesman|   50000|Barcelona|
|        6|     widgeteer|   35000|   Madrid|
|        7|     widgeteer|   35000|Barcelona|
|        8|    psychopath|  100000|   Madrid|
|        9|     widgeteer|   35000|Barcelona|
|       10|     widgeteer|   35000|   Madrid|
|       11|     widgeteer|   35000|Barcelona|
|       12|smoke salesman|   50000|   Madrid|
|       13|smoke salesman|   50000|Barcelona|
|       14|     widgeteer|   35000|   Madrid|
|       15|smoke salesman|   50000|Barcelona|
|       16|    psychopath|  100000|   Madrid|
|       17|smoke salesman|   50000|Barcelona|
|       18|    psychopath|  100000

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

In [383]:
session.sql('''SELECT * from df''')

AnalysisException: 'Table or view not found: df; line 1 pos 14'

In [384]:
df.registerTempTable('df_table')

Once registered, we can perform queries as complex as we want.

In [388]:
session.sql('''SELECT * from df_table 
               WHERE location="Madrid" AND salary>40000''').show()

+---------+--------------+------+--------+
|id_number|      position|salary|location|
+---------+--------------+------+--------+
|        3|smoke salesman| 50000|  Madrid|
|        4|smoke salesman| 50000|  Madrid|
|        8|    psychopath|100000|  Madrid|
|       12|smoke salesman| 50000|  Madrid|
|       16|    psychopath|100000|  Madrid|
|       18|    psychopath|100000|  Madrid|
+---------+--------------+------+--------+



In [390]:
session.sql('''SELECT id_number, log1p(id_number) from df_table 
               WHERE location="Madrid" AND salary>40000''').show()

+---------+--------------------------------+
|id_number|LOG1P(CAST(id_number AS DOUBLE))|
+---------+--------------------------------+
|        3|              1.3862943611198906|
|        4|              1.6094379124341003|
|        8|              2.1972245773362196|
|       12|              2.5649493574615367|
|       16|               2.833213344056216|
|       18|              2.9444389791664403|
+---------+--------------------------------+



In [395]:
def classist(salary):
    return 'perroflauta' if salary < 42000 else 'burgues'

class_udf = functions.udf(classist)

df.select('id_number', 'position', class_udf('salary')).show()

+---------+--------------+----------------+
|id_number|      position|classist(salary)|
+---------+--------------+----------------+
|        0|     widgeteer|     perroflauta|
|        1|     widgeteer|     perroflauta|
|        2|        wizard|         burgues|
|        3|smoke salesman|         burgues|
|        4|smoke salesman|         burgues|
|        5|smoke salesman|         burgues|
|        6|     widgeteer|     perroflauta|
|        7|     widgeteer|     perroflauta|
|        8|    psychopath|         burgues|
|        9|     widgeteer|     perroflauta|
|       10|     widgeteer|     perroflauta|
|       11|     widgeteer|     perroflauta|
|       12|smoke salesman|         burgues|
|       13|smoke salesman|         burgues|
|       14|     widgeteer|     perroflauta|
|       15|smoke salesman|         burgues|
|       16|    psychopath|         burgues|
|       17|smoke salesman|         burgues|
|       18|    psychopath|         burgues|
|       19|        wizard|      

In [396]:
session.sql('''SELECT id_number, position, class_udf(salary) 
               FROM df_table 
               WHERE location="Madrid" AND salary>40000''').show()

AnalysisException: "Undefined function: 'class_udf'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 28"

In [398]:
session.udf.register('classist_sql', class_udf)

session.sql('''SELECT id_number, position, classist_sql(salary)  
               FROM df_table 
               WHERE location="Madrid"''').show()


+---------+--------------+--------------------+
|id_number|      position|classist_sql(salary)|
+---------+--------------+--------------------+
|        0|     widgeteer|         perroflauta|
|        3|smoke salesman|             burgues|
|        4|smoke salesman|             burgues|
|        6|     widgeteer|         perroflauta|
|        8|    psychopath|             burgues|
|       10|     widgeteer|         perroflauta|
|       12|smoke salesman|             burgues|
|       14|     widgeteer|         perroflauta|
|       16|    psychopath|             burgues|
|       18|    psychopath|             burgues|
+---------+--------------+--------------------+



#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

In [401]:
pandas_df = df.toPandas()
pandas_df.head()

,id_number,position,salary,location
0,0,widgeteer,35000.0,Madrid
1,1,widgeteer,35000.0,Barcelona
2,2,wizard,60000.0,Barcelona
3,3,smoke salesman,50000.0,Madrid
4,4,smoke salesman,50000.0,Madrid


In [402]:
session.createDataFrame(pandas_df)

DataFrame[id_number: bigint, position: string, salary: double, location: string]

## Writing out


In [403]:
df.write.csv('df.csv')

#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

In [412]:
df.sort('salary', ascending=False).show()

+---------+--------------+--------+---------+
|id_number|      position|  salary| location|
+---------+--------------+--------+---------+
|      100|     superboss|12000000|     null|
|       16|    psychopath|  100000|   Madrid|
|       18|    psychopath|  100000|   Madrid|
|        8|    psychopath|  100000|   Madrid|
|      101|          null|  100000|    Miami|
|       19|        wizard|   60000|Barcelona|
|        2|        wizard|   60000|Barcelona|
|       13|smoke salesman|   50000|Barcelona|
|       12|smoke salesman|   50000|   Madrid|
|        3|smoke salesman|   50000|   Madrid|
|       17|smoke salesman|   50000|Barcelona|
|        4|smoke salesman|   50000|   Madrid|
|        5|smoke salesman|   50000|Barcelona|
|       15|smoke salesman|   50000|Barcelona|
|        9|     widgeteer|   35000|Barcelona|
|        7|     widgeteer|   35000|Barcelona|
|        0|     widgeteer|   35000|   Madrid|
|       10|     widgeteer|   35000|   Madrid|
|        1|     widgeteer|   35000

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

In [418]:
!head -n 2 coupon150720.csv

79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0
79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0


In [423]:
df = session.sql('''SELECT 
                      _c0 AS tkt_number,
                      _c1 AS cpn_number, 
                      _c2 AS origin, 
                      _c3 AS destination, 
                      _c4 AS airline, 
                      CAST(_c6 AS FLOAT) AS amount 
                    FROM csv.`coupon150720.csv`''')
df

DataFrame[tkt_number: string, cpn_number: string, origin: string, destination: string, airline: string, amount: float]

In [425]:
df.where(df['destination'] == 'MAD').head()

Row(tkt_number='79062005639642', cpn_number='4', origin='BRU', destination='MAD', airline='UX', amount=21.020000457763672)

2) Total ticket amounts per origin

In [431]:
gd = df.where(df['destination'] == 'MAD').groupby('origin')
gd.sum('amount')\
  .sort('sum(amount)', ascending=False)\
  .show(20)

+------+------------------+
|origin|       sum(amount)|
+------+------------------+
|   CCS| 94528.67986679077|
|   GRU| 87192.63982868195|
|   EZE| 81074.64043807983|
|   BOG| 74644.44981968403|
|   LHR| 69609.53006839752|
|   LPA|60483.920072078705|
|   MEX|56316.729888916016|
|   JFK|53496.170244693756|
|   TLV| 53436.21986293793|
|   TFN| 50034.94993495941|
|   LIM|48328.139808654785|
|   MIA|45660.920145988464|
|   SCL|  42584.9598236084|
|   PMI| 40547.17005729675|
|   FRA| 38863.54004433751|
|   MUC| 37186.16008067131|
|   DOH|34534.209548950195|
|   BCN|32060.600192189217|
|   FCO|30036.770029067993|
|   AMS|29872.270063877106|
+------+------------------+
only showing top 20 rows



3) Top 10 Airlines by average amount



In [432]:
gd = df.where(df['destination'] == 'MAD').groupby('airline')
gd.mean('amount')\
  .sort('avg(amount)', ascending=False)\
  .show(20)

AnalysisException: "cannot resolve '`mean(amount)`' given input columns: [airline, avg(amount)];;\n'Sort ['mean(amount) DESC NULLS LAST], true\n+- AnalysisBarrier\n      +- Aggregate [airline#5871], [airline#5871, avg(cast(amount#5872 as double)) AS avg(amount)#6007]\n         +- Filter (destination#5870 = MAD)\n            +- Project [_c0#5883 AS tkt_number#5867, _c1#5884 AS cpn_number#5868, _c2#5885 AS origin#5869, _c3#5886 AS destination#5870, _c4#5887 AS airline#5871, cast(_c6#5889 as float) AS amount#5872]\n               +- Relation[_c0#5883,_c1#5884,_c2#5885,_c3#5886,_c4#5887,_c5#5888,_c6#5889,_c7#5890,_c8#5891,_c9#5892,_c10#5893,_c11#5894,_c12#5895,_c13#5896,_c14#5897] csv\n"

## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf